In [4]:
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.linear_model import LinearRegression, LogisticRegression, RidgeClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, StackingClassifier
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
# from jupyterthemes import jtplot
from sklearn.metrics import auc, roc_curve, roc_auc_score
# %matplotlib inline
# jtplot.style()

In [5]:
df = pd.read_csv('train.csv')

In [6]:
df.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [8]:
X = df.drop('SalePrice', axis=1)
y_val = df.SalePrice
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [9]:
#Находим категориальные признаки
cat_feat = list(X.dtypes[X.dtypes == object].index)

#закодируем пропущенные значений строкой, факт пропущенного значения тоже может нести в себе информацию
# X[cat_feat] = X[cat_feat].fillna('nan')

#отфильтруем непрерывные признаки
num_feat = [f for f in X if f not in (cat_feat + ['Id'])]

# Смотрим сколько у нас значений по каждому категориальному признаку
cat_nunique = X[cat_feat].nunique()
# print(cat_nunique)

# В этом датасете все признаки с менее чем 30ю уникальными значениями, проверку ниже пропустим
# #Чтобы в разы не увеличивать число признаков при построении dummy,
# #будем использовать категориальные признаки с < 30 уникальных значений
# cat_feat = list(cat_nunique[cat_nunique < 30].index) # 

In [10]:
# Создаем дамми-переменные для категорий
X_dummies = pd.get_dummies(X[cat_feat], columns=cat_feat)
# dummy_test = pd.get_dummies(X_test[cat_feat], columns=cat_feat)

dummy_cols = list(set(X_dummies))

X_dummies = X_dummies[dummy_cols]
# dummy_test = dummy_test[dummy_cols]

# Заменяем пропуски на специальное значение -999, чтобы деревья могли их отличить
X = pd.concat([X[num_feat].fillna(-999),
                     X_dummies], axis=1)

# X_test = pd.concat([d_test[num_feat].fillna(-999),
#                      dummy_test], axis=1)

In [11]:
X.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,Neighborhood_Crawfor,RoofStyle_Mansard,Neighborhood_Sawyer,Heating_GasW,MiscFeature_Othr,Exterior2nd_Brk Cmn,GarageFinish_RFn,GarageType_BuiltIn,Functional_Min2,Neighborhood_Blmngtn
0,60,65.0,8450,7,5,2003,2003,196.0,706,0,...,0,0,0,0,0,0,1,0,0,0
1,20,80.0,9600,6,8,1976,1976,0.0,978,0,...,0,0,0,0,0,0,1,0,0,0
2,60,68.0,11250,7,5,2001,2002,162.0,486,0,...,0,0,0,0,0,0,1,0,0,0
3,70,60.0,9550,7,5,1915,1970,0.0,216,0,...,1,0,0,0,0,0,0,0,0,0
4,60,84.0,14260,8,5,2000,2000,350.0,655,0,...,0,0,0,0,0,0,1,0,0,0


In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y_val, test_size=0.33, random_state=42)

In [19]:
from sklearn.ensemble import RandomForestRegressor
rf_regressor = RandomForestRegressor()
rf_regressor.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [20]:
from sklearn.metrics import r2_score
r2_score(y_test, rf_regressor.predict(X_test))

0.8659614662931052

In [16]:
# Есть неважные признаки. Можно будет отбрасывать, если понадобится ускорение (и для стабильности работы)
imp = pd.Series(rf_regressor.feature_importances_)
imp.sort_values(ascending=False)

3      0.658183
15     0.136271
11     0.031714
12     0.030117
8      0.029251
         ...   
179    0.000000
178    0.000000
177    0.000000
176    0.000000
143    0.000000
Length: 288, dtype: float64

In [17]:
# линейная регрессия отдельно
lreg_model = LinearRegression()
lreg_model.fit(X_train, y_train)
r2_score(y_test, lreg_model.predict(X_test))

0.8733850919299049

In [21]:
# у логистической регрессии - плохо
logreg_model = LogisticRegression()
logreg_model.fit(X_train, y_train)
r2_score(y_test, logreg_model.predict(X_test))

C:\Users\Sergei\anaconda3\envs\mw-flow-calculator\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.3762670486967731

In [23]:
from sklearn.tree import DecisionTreeRegressor
dt_regressor = DecisionTreeRegressor()
dt_regressor.fit(X_train, y_train)
# from sklearn.metrics import r2_score
r2_score(y_test, dt_regressor.predict(X_test))

0.6326956724105031

In [31]:
from sklearn.ensemble import StackingRegressor
regressor = StackingRegressor(
    [
        ('lr', RandomForestRegressor()),
        ('linreg', LinearRegression()),
        ('dt', DecisionTreeRegressor()),
        # ('rf', LogisticRegression())
    ],
final_estimator = LinearRegression())

In [33]:
# Результат чуть лучше, чем у отдельного дерева. 
regressor.fit(X_train, y_train)
r2_score(y_test, regressor.predict(X_test))

0.8813549804799786